In [1]:
import pandas as pd
import numpy as np

In [2]:
BX_Book_Ratings = pd.read_csv('BX-Book-Ratings.csv',encoding = 'cp437',sep = ';')

#BX_Books = pd.read_csv('BX-Books.csv',encoding = 'cp437',sep = ';')

#BX_Users = pd.read_csv('BX-Users.csv',encoding = 'cp437',sep = ';')

In [3]:
print(BX_Book_Ratings.shape)
BX_Book_Ratings.head()

(1149780, 3)


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
# NULL값이 없는 것 확인
BX_Book_Ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [5]:
# ID 개수, 책 개수
print(BX_Book_Ratings['User-ID'].nunique())
print(BX_Book_Ratings['ISBN'].nunique())

105283
340556


In [6]:
BX_Book_Ratings['ISBN'].nunique()

340556

In [8]:
def Euclidean_Distance(data, ID1, ID2):
    def EUC(data):    
        # 두 개의 rating의 차
        return data.iloc[0] - data.iloc[1]
        
    # 비교하고 싶은 두 User의 데이터
    combine = data[(data['User-ID'] == ID1) | (data['User-ID'] == ID2)]
    
    # User별 책에대한 Rating
    result_data = pd.pivot_table(combine, index = 'User-ID',columns = 'ISBN', values = 'Book-Rating')
    
    # 두 개의 ID의 책 Rating이 하나라도 nan이면 제거
    distance = result_data.apply(EUC).dropna(how = 'any')
    
    #print(distance)
    
    # 각 rating의 차를 제곱해 sum한 값의 제곱근을 return
    return pow((np.array(distance)**2).sum(),1/2)

In [9]:
# 원하는 ID 2개를 입력하면 Euclidean_Distance가 나온다.
Euclidean_Distance(BX_Book_Ratings, 242106, 11676)

68.78226515607058

In [10]:
# 원하는 ID 2개를 입력하면 Euclidean_Distance가 나온다.
Euclidean_Distance(BX_Book_Ratings, 1113, 11676)

0.0

In [11]:
def Cosine_similarity(data, ID1, ID2):
    def COS(data):    
        # 두 개의 rating의 곱
        return data.iloc[0] * data.iloc[1]
        
    # 비교하고 싶은 두 User의 데이터
    combine = data[(data['User-ID'] == ID1) | (data['User-ID'] == ID2)]
    
    # User별 책에대한 Rating
    result_data = pd.pivot_table(combine, index = 'User-ID',columns = 'ISBN', values = 'Book-Rating')
    
    # 두 개의 ID의 책 Rating이 하나라도 nan이면 제거
    distance = result_data.dropna(how = 'any', axis = 1)
    
    # 각 ID별 제곱한 값을 합하고 루트 씌워주기
    ID_squared_sum = pow((np.array(distance)**2).sum(axis = 1),1/2)
    
    print(ID_squared_sum.sum())
    
    # ID별 rating값의 곱의 합
    ID_multiply_sum = distance.apply(COS).sum()
    
    print(ID_multiply_sum)
    
    # inf값이 나올 때 1,-1로 바꿔줌
    if ID_squared_sum.sum():
        result = ID_multiply_sum / (ID_squared_sum[0] * ID_squared_sum[1])
    else:
        if ID_squared_sum[0] and ID_squared_sum[1]:
            result = -1
        else:
            result = 1
    
    return result

In [12]:
# 원하는 ID 2개를 입력하면 Cosine_similarity 나온다.
Cosine_similarity(BX_Book_Ratings, 242106, 11676)

118.15157175160785
1207.0


0.35422934762376884

In [13]:
# 원하는 ID 2개를 입력하면 Cosine_similarity 나온다.
Cosine_similarity(BX_Book_Ratings, 1113, 11676)

0.0
0.0


1

In [14]:
def Pearson_Correlation_Coefficient(data, ID1, ID2):
    def PCC_subtract(data):    
        # 각 ID의 rating값에서 평균 빼주기
        return (data - ID_mean)
    
    def PCC_multiply(data):
        # 서로 rating값끼리 곱해주기
        return data.iloc[0] * data.iloc[1]
        
    # 비교하고 싶은 두 User의 데이터
    combine = data[(data['User-ID'] == ID1) | (data['User-ID'] == ID2)]
    
    # User별 책에대한 Rating
    result_data = pd.pivot_table(combine, index = 'User-ID',columns = 'ISBN', values = 'Book-Rating')
    
    # 각 ID별 평균 구하기
    ID_mean = result_data.mean(axis = 1)
    
    print(ID_mean)
    
    # 두 개의 ID의 책 Rating이 하나라도 nan이면 제거
    distance = result_data.dropna(how = 'any', axis = 1)
    
    # 각 rating에 평균을 빼주고 서로 곱한 값의 합
    ID_multiply_sum = distance.apply(PCC_subtract).apply(PCC_multiply).sum()
    
    print(ID_multiply_sum)
    
    # 각 ID의 rating 값을 제곱한 값을 합하고 루트 씌워주기
    ID_sqrt_squared_sum = pow((np.array(distance.apply(PCC_subtract))**2).sum(axis=1),1/2)
    
    print(ID_sqrt_squared_sum)
    
    return ID_multiply_sum / (ID_sqrt_squared_sum[0] * ID_sqrt_squared_sum[1])

In [15]:
# 원하는 ID 2개를 입력하면 Cosine_similarity 나온다.
Pearson_Correlation_Coefficient(BX_Book_Ratings, 242106, 11676)

User-ID
11676     4.564917
242106    1.392505
dtype: float64
-42.00919664035956
[42.5398098  44.73167428]


-0.02207667573628445

In [16]:
# 원하는 ID 2개를 입력하면 Cosine_similarity 나온다.
Pearson_Correlation_Coefficient(BX_Book_Ratings, 1113, 11676)

User-ID
1113     2.800000
11676    4.564917
dtype: float64
12.78176738714895
[2.8        4.56491692]


1.0

In [17]:
#BX_Book_Ratings[BX_Book_Ratings['ISBN'] == BX_Book_Ratings['ISBN'][13]]